### Reporting days spent on development activity category in given month using Story Points

In [ ]:
from jira import JIRA
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from dateutil.rrule import rrule, MONTHLY
from datetime import datetime
import calendar
import datetime

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

jira_url = 'https://kainos-evolve.atlassian.net'
jira = JIRA(jira_url)

In [ ]:
issues = pd.DataFrame()
issues['key'] = ''
issues['category'] = ''
issues['type'] = ''
issues['team'] = ''
issues['SP'] = ''
issues['bugEffort'] = ''
issues['status'] = ''
issues['summary'] = ''
issues['month'] = ''

In [ ]:
year = 2018

In [ ]:
# download data for each month of a year defined previously
for i in range(1,13):
    lastDay = calendar.monthrange(year,i)[1]
    #don't query Jira for months in the future
    now = datetime.datetime.now()
    if i > now.month: break
    
    jql = 'project = vxt and status = Completed AND status changed to Completed during ({}-{}-1, {}-{}-{}) \
 AND type in (bug, "External Bug", story)'.format(year, i, year, i, lastDay)

    issuesRaw = jira.search_issues(jql, maxResults = False)
    jql
    for issue in issuesRaw:

        try:
            bugEffort = issue.fields.customfield_14703
        except AttributeError:
            bugEffort = np.nan

        try:
            team = str(issue.fields.customfield_14200.value)
        except AttributeError:
            team = np.nan

        try:
            category = str(issue.fields.customfield_14709.value)
        except AttributeError:
            category = 'Not set'

        issues = issues.append(
            {
             'key': issue.key,
             'type': issue.fields.issuetype.name,
             'status': issue.fields.status.name,
             'category': issue.fields.customfield_14709,
             'bugEffort': bugEffort,
             'SP' : issue.fields.customfield_10005,
             'summary': issue.fields.summary,
             'team': team,
             'month': i
            }, ignore_index=True)



In [ ]:
#use bug effort if SP is empty for bug type issues
issues.loc[(issues['type'] == 'Bug') & (issues['SP'].isnull()), ["SP"]] = issues.bugEffort

#filter teams
issues = issues.loc[(issues['team'].isin(["Gdansk Team 1", "Gdansk Team 2", "Belfast Team"]))]


In [ ]:
issues['SP']=issues['SP'].fillna(0)

'not estimated issues or issues with zero SP'
notEstimated = issues.loc[(issues['type'] == 'Story') & (issues['SP'] == 0)]
notEstimated


#issues['category']=issues['category'].fillna('Not set')
issues['category'] = issues.category.astype(str)



In [ ]:
#group by by month and category and sum SP
issuesByCategory = issues.groupby(['month', 'category']).agg({'SP':'sum'})

#add column with SP % per month
issuesByCategory['SPPerc'] = issuesByCategory.groupby(level=[0]).apply(lambda x: x / float(x.sum()))
#reindexing needed to relate month to each row 
issuesByCategory = issuesByCategory.reset_index()

#days from Kimble used for calculations
monthDevDays = [100, 100]

#first add days from monthDevDays for specified month
issuesByCategory['Days'] = issuesByCategory['month'].apply(lambda x: monthDevDays[x-1])
#then multiply by % of month spent on category
issuesByCategory['Days'] = issuesByCategory['Days'] * issuesByCategory['SPPerc']
#round values
issuesByCategory = issuesByCategory.round({'SPPerc': 2, 'Days':2})

#
issuesByCategory
